In [ ]:
from gensim.models import Word2Vec
from sklearn.model_selection import ParameterGrid
import pandas as pd
import h5py
import os
import logging

In [ ]:
def show_synonyms(model, search_str, num_synonyms, verbose=True):
    synonym_list = list()
    movie_index = df_movies[df_movies[TITLE].str.match(search_str)]
    for mi in movie_index.index:
        synonym_list.extend([(i, df_movies.loc[int(i[0])][TITLE]) for i in 
                             list(model.wv.most_similar(str(mi), topn=num_synonyms))])
    cosine_similarity = pd.Series([i[0][1] for i in synonym_list])
    mean = cosine_similarity.mean()
    stddev = cosine_similarity.std()
    if verbose:
        print(movie_index)
        print('Mean: {} \t StdDev: {}'.format(mean, stddev))
    return synonym_list, mean, stddev

In [ ]:
MOVIE_TARGETS = '|'.join([
    '(^Matrix, The)',
    '(Saving Private Ryan)',
    '(Lawrence of Arabia)',
    '(Bridesmaids)',
    '(^Notebook, The \(2004)',
])

In [ ]:
MOVIE_ID = 'movieId'
TITLE = 'title'

In [ ]:
df_movies = pd.read_csv('ml-20m/movies.csv', index_col=MOVIE_ID)

In [ ]:
files = [f for f in os.listdir() if f[-7:] == '.gensim']

In [ ]:
results = pd.DataFrame([[f] + list(show_synonyms(Word2Vec.load(f), MOVIE_TARGETS, 5, False)[1:]) for f in files], columns=['file', 'mean', 'stddev'])

In [ ]:
results.sort_values(by=['mean'])

In [ ]:
# w2v_vs_64_sg_0_hs_1_mc_1_it_1_wn_5774_ng_0.gensim not horrible, not great max ~0.7
# w2v_vs_64_sg_1_hs_1_mc_1_it_1_wn_5774_ng_0.gensim on point, quite good, max ~0.9
# w2v_vs_64_sg_0_hs_1_mc_1_it_1_wn_5774_ng_2.gensim not horrible, not great max ~0.64
# w2v_vs_64_sg_0_hs_0_mc_1_it_1_wn_5774_ng_0.gensim almost junk, max ~0.47
# w2v_vs_64_sg_1_hs_0_mc_1_it_1_wn_5774_ng_0.gensim almost junk max ~0.47
# w2v_vs_64_sg_1_hs_0_mc_1_it_1_wn_5774_ng_2.gensim super good max ~0.95
# w2v_vs_64_sg_1_hs_1_mc_1_it_1_wn_5774_ng_2.gensim super good max ~0.95
# w2v_vs_64_sg_0_hs_0_mc_1_it_1_wn_5774_ng_2.gensim mixed bag max ~0.86
model = Word2Vec.load('w2v_vs_64_sg_0_hs_1_mc_1_it_1_wn_5774_ng_0.gensim')
show_synonyms(model, MOVIE_TARGETS, 5)